In [1]:
import torch
import transformers
import trl
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

In [2]:
from huggingface_hub import login

login(
  token="y",  
  add_to_git_credential=True
)


Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir= '/workspace/gemma-2-9b-it',
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
    load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# 데이터

In [4]:
dataset = load_dataset("json", data_files="/workspace/train_dataset.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"].select(range(2000))
test_dataset = dataset["test"].select(range(20))

In [5]:

train_valid= train_dataset.train_test_split(test_size=0.1, seed=45)
train_dataset = train_valid["train"].select(range(20))
valid_dataset = train_valid["test"].select(range(20))

In [6]:
train_dataset

Dataset({
    features: ['messages'],
    num_rows: 20
})

In [7]:
def template_dataset(examples):
    return{"text":  tokenizer.apply_chat_template(examples["messages"], tokenize=False)}

train_dataset = train_dataset.map(template_dataset, remove_columns=["messages"])
valid_dataset = valid_dataset.map(template_dataset, remove_columns=["messages"])
train_dataset[0]

{'text': '<bos><start_of_turn>user\n안녕하세요.<end_of_turn>\n<start_of_turn>model\n요즘 어떤 고민이 있으신가요?<end_of_turn>\n<start_of_turn>user\n요즘 일이 너무 바쁜데 일이 줄어들지 않아요. 회사에서 퇴근하고 집에와서도 또 일이 생기고 하는 것 같아요. 일이 줄어들지 않아서 힘들고 지치는데 이러다가 우울증이 올 것 같아서 걱정이 되네요. 어떻게 해야 할까요?<end_of_turn>\n<start_of_turn>model\n이해합니다. 일이 많아져서 지치고 우울해지는 것은 어렵고 고통스러운 일입니다. 어떤 일 때문에 힘들어지는지, 구체적으로 말씀해주실 수 있나요?<end_of_turn>\n<start_of_turn>user\n회사에서 일이 생길 때마다 또 다른 일이 빠듯해서 일이 줄어들지 않아요.<end_of_turn>\n<start_of_turn>model\n현재에는 일과 휴식의 균형이 조화롭지 않아 스트레스가 쌓이고, 지치고 우울해지는 것 같습니다. 스트레스는 몸과 마음에 나쁜 영향을 미치기 때문에, 일을 줄이고 휴식을 취해야 합니다. 하지만 일을 줄이기 어렵다면, 일을 한 뒤 적당한 운동과 힐링 타임을 가져보시면 어떨까요? 무리하지 않고 즐길 수 있는 취미도 추천해드립니다. 일과 취미, 여가활동의 균형을 유지해 보시는 것이 좋습니다.<end_of_turn>\n'}

In [8]:
train_dataset[0]

{'text': '<bos><start_of_turn>user\n안녕하세요.<end_of_turn>\n<start_of_turn>model\n요즘 어떤 고민이 있으신가요?<end_of_turn>\n<start_of_turn>user\n요즘 일이 너무 바쁜데 일이 줄어들지 않아요. 회사에서 퇴근하고 집에와서도 또 일이 생기고 하는 것 같아요. 일이 줄어들지 않아서 힘들고 지치는데 이러다가 우울증이 올 것 같아서 걱정이 되네요. 어떻게 해야 할까요?<end_of_turn>\n<start_of_turn>model\n이해합니다. 일이 많아져서 지치고 우울해지는 것은 어렵고 고통스러운 일입니다. 어떤 일 때문에 힘들어지는지, 구체적으로 말씀해주실 수 있나요?<end_of_turn>\n<start_of_turn>user\n회사에서 일이 생길 때마다 또 다른 일이 빠듯해서 일이 줄어들지 않아요.<end_of_turn>\n<start_of_turn>model\n현재에는 일과 휴식의 균형이 조화롭지 않아 스트레스가 쌓이고, 지치고 우울해지는 것 같습니다. 스트레스는 몸과 마음에 나쁜 영향을 미치기 때문에, 일을 줄이고 휴식을 취해야 합니다. 하지만 일을 줄이기 어렵다면, 일을 한 뒤 적당한 운동과 힐링 타임을 가져보시면 어떨까요? 무리하지 않고 즐길 수 있는 취미도 추천해드립니다. 일과 취미, 여가활동의 균형을 유지해 보시는 것이 좋습니다.<end_of_turn>\n'}

In [9]:
instruction_template = "user\n"
response_template = "model\n"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)


# 학습

In [10]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"],
        task_type="CAUSAL_LM",
)

args = TrainingArguments(
    output_dir="./model_output", 
    num_train_epochs=1,          
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,  
    optim="adamw_torch_fused",    
    logging_steps=2,            
    eval_strategy="steps",
    eval_steps=2,
    save_steps =5,
    learning_rate=2e-4,           
    bf16=True,                    
    tf32=True,                    
    max_grad_norm=0.3,            
    warmup_ratio=0.03,            
    lr_scheduler_type="constant", 
    #push_to_hub=True,             
    report_to="wandb",            
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    max_seq_length=512,
    peft_config=peft_config,
    tokenizer=tokenizer,
    #packing=True,
    dataset_text_field="text",
    data_collator=collator,

)
trainer.train()


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: wooseoksong (wooseoksong-study). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 d

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.71 GiB. GPU 0 has a total capacity of 23.67 GiB of which 1.50 GiB is free. Process 2481689 has 22.16 GiB memory in use. Of the allocated memory 21.24 GiB is allocated by PyTorch, and 676.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)